In [78]:
from typing import Any, Dict

import numpy as np

from game_mechanics import GoEnv, choose_move_randomly, load_pkl, play_go, save_pkl

from torch import nn
import torch
import torch.nn.functional as F
from tqdm import tqdm_notebook

from random import choice
from einops.layers.torch import Rearrange
from einops import rearrange


In [42]:
def random_move(observation, legal_moves):
    return choice(legal_moves)

In [52]:
env = GoEnv(opponent_choose_move = random_move)

In [55]:
board, reward, done, dict_ = env.reset()
legal_moves = dict_['legal_moves']

In [56]:
for _ in range(10):
    board, reward, done, dict_ = env.step(choice(legal_moves))

In [57]:
board

array([[ 0,  0,  0,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  1, -1,  1, -1,  0, -1],
       [ 0,  0,  0, -1,  0,  0,  0,  0,  1],
       [-1,  1, -1,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  1,  0,  0,  0,  0,  0,  0, -1],
       [ 0,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, -1,  0, -1, -1,  0,  0]])

In [124]:
asd = flat(t_board)
asd[0,[1,2,3]] = -torch.inf
sdf = F.softmax(asd)

C:\Users\hristo\AppData\Local\Temp\ipykernel_9424\1780493828.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  sdf = F.softmax(asd)


In [126]:
sdf.detach().numpy()

array([[0.01125345, 0.        , 0.        , 0.        , 0.01125345,
        0.01125345, 0.01125345, 0.01125345, 0.01125345, 0.01125345,
        0.01125345, 0.03059006, 0.03059006, 0.00413991, 0.03059006,
        0.00413991, 0.01125345, 0.00413991, 0.01125345, 0.01125345,
        0.01125345, 0.00413991, 0.01125345, 0.01125345, 0.01125345,
        0.01125345, 0.03059006, 0.00413991, 0.03059006, 0.00413991,
        0.01125345, 0.01125345, 0.01125345, 0.01125345, 0.01125345,
        0.03059006, 0.01125345, 0.01125345, 0.03059006, 0.01125345,
        0.01125345, 0.01125345, 0.01125345, 0.01125345, 0.01125345,
        0.01125345, 0.01125345, 0.01125345, 0.01125345, 0.01125345,
        0.01125345, 0.01125345, 0.01125345, 0.03059006, 0.01125345,
        0.03059006, 0.01125345, 0.01125345, 0.01125345, 0.01125345,
        0.01125345, 0.01125345, 0.00413991, 0.01125345, 0.03059006,
        0.01125345, 0.01125345, 0.01125345, 0.01125345, 0.01125345,
        0.01125345, 0.01125345, 0.01125345, 0.01

In [116]:
F.softmax(flat(t_board))

C:\Users\hristo\AppData\Local\Temp\ipykernel_9424\97766271.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(flat(t_board))


tensor([[0.0107, 0.0107, 0.0107, 0.0290, 0.0107, 0.0107, 0.0107, 0.0107, 0.0107,
         0.0107, 0.0107, 0.0290, 0.0290, 0.0039, 0.0290, 0.0039, 0.0107, 0.0039,
         0.0107, 0.0107, 0.0107, 0.0039, 0.0107, 0.0107, 0.0107, 0.0107, 0.0290,
         0.0039, 0.0290, 0.0039, 0.0107, 0.0107, 0.0107, 0.0107, 0.0107, 0.0290,
         0.0107, 0.0107, 0.0290, 0.0107, 0.0107, 0.0107, 0.0107, 0.0107, 0.0107,
         0.0107, 0.0107, 0.0107, 0.0107, 0.0107, 0.0107, 0.0107, 0.0107, 0.0290,
         0.0107, 0.0290, 0.0107, 0.0107, 0.0107, 0.0107, 0.0107, 0.0107, 0.0039,
         0.0107, 0.0290, 0.0107, 0.0107, 0.0107, 0.0107, 0.0107, 0.0107, 0.0107,
         0.0107, 0.0107, 0.0107, 0.0039, 0.0107, 0.0039, 0.0039, 0.0107, 0.0107]])

In [104]:
def tensorize(board):
    return torch.as_tensor(rearrange(board, 'w h -> 1 w h', w=9, h=9), dtype=torch.float32)

policy = nn.Sequential(
    nn.Conv2d(1, 10, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(10, 1, kernel_size=1, padding=0),
    Rearrange('b 1 w h -> b (w h)', w=9, h=9),
    nn.Softmax(dim=1)
)

V = nn.Sequential(
    nn.Conv2d(1, 10, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(10, 1, kernel_size=1, padding=0),
    Rearrange('b 1 w h -> b (w h)', w=9, h=9),
    nn.Linear(81, 1)
)

t_board = rearrange(tensorize(board), '1 w h -> 1 1 w h')
policy(t_board)

In [105]:
t_board = rearrange(tensorize(board), '1 w h -> 1 1 w h')
policy(t_board)

tensor([[0.0126, 0.0106, 0.0104, 0.0138, 0.0136, 0.0107, 0.0160, 0.0128, 0.0144,
         0.0126, 0.0138, 0.0157, 0.0128, 0.0111, 0.0118, 0.0120, 0.0095, 0.0092,
         0.0125, 0.0118, 0.0141, 0.0107, 0.0110, 0.0135, 0.0104, 0.0111, 0.0094,
         0.0126, 0.0103, 0.0092, 0.0120, 0.0128, 0.0126, 0.0126, 0.0145, 0.0138,
         0.0118, 0.0135, 0.0112, 0.0131, 0.0126, 0.0126, 0.0126, 0.0112, 0.0119,
         0.0106, 0.0110, 0.0146, 0.0122, 0.0126, 0.0126, 0.0126, 0.0138, 0.0144,
         0.0123, 0.0108, 0.0137, 0.0126, 0.0126, 0.0126, 0.0126, 0.0122, 0.0127,
         0.0145, 0.0138, 0.0130, 0.0144, 0.0128, 0.0147, 0.0125, 0.0103, 0.0108,
         0.0135, 0.0138, 0.0112, 0.0116, 0.0110, 0.0111, 0.0110, 0.0120, 0.0126]],
       grad_fn=<SoftmaxBackward0>)

In [108]:
t_board = rearrange(tensorize(board), '1 w h -> 1 1 w h')
V(t_board)

tensor([[0.1481]], grad_fn=<AddmmBackward0>)